In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
from pyrosm import OSM, get_data
import geopandas as gpd
import pandas as pd
from sklearn.neighbors import BallTree
import numpy as np
import mapclassify as mc
# import matplotlib.pyplot as plt
import time
import networkx as nx
import igraph as ig
import osmnx as ox
import folium
import json
from datetime import datetime, timedelta
from folium import PolyLine, Marker
from folium.plugins import BeautifyIcon
from math import radians, cos, sin, asin, sqrt
import random as r
import math


In [3]:
# Coordenadas para Santiago Centro
lat, lon = -33.456574386393314, -70.65970917320084

# Ajusta los límites usando el valor de delta
delta = 0.09  # Menor valor para un área más pequeña
north = lat + delta
south = lat - delta
east = lon + delta
west = lon - delta

# Obtiene el grafo usando los límites ajustados
G = ox.graph_from_bbox(north, south, east, west, network_type='drive')
G = ox.speed.add_edge_speeds(G)
G = ox.speed.add_edge_travel_times(G)

almacen = {"id": "0", "lat": "-33.456574386393314", "long": "-70.65970917320084", "demand": 0}

# Datos de los clientes
with open("nodos.json", 'r', encoding='utf-8') as archivo:
    nodos = json.load(archivo)


nodos.insert(0, almacen)  # Asegúrate de que el almacén es el primer nodo en la lista
# r.seed(42)
# np.random.seed(42)

In [4]:
def distancia_camion(nodo1, nodo2, G):
    source_node = ox.distance.nearest_nodes(G, X=float(nodo1['long']), Y=float(nodo1['lat']))
    target_node = ox.distance.nearest_nodes(G, X=float(nodo2['long']), Y=float(nodo2['lat']))
    ruta = ox.shortest_path(G, source_node, target_node, weight='length')
    return sum(ox.utils_graph.get_route_edge_attributes(G, ruta, 'length'))

def haversine(lat1, lon1, lat2, lon2):
    # Convertir coordenadas de grados a radianes
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Fórmula de Haversine
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371 # Radio de la Tierra en kilómetros
    return c * r

def distancia_dron(nodo1, nodo2):
    return haversine(float(nodo1['lat']), float(nodo1['long']), float(nodo2['lat']), float(nodo2['long']))

def matriz_distancias_camion(lista_nodos, G):
    cant_nodos = len(lista_nodos)
    matriz_distancias_camion = [[0 for _ in range(cant_nodos)] for _ in range(cant_nodos)]
    for i in range(cant_nodos):
        for j in range(i + 1, cant_nodos):
            distancia = distancia_camion(lista_nodos[i], lista_nodos[j], G)
            matriz_distancias_camion[i][j] = matriz_distancias_camion[j][i] = distancia/1000
    return matriz_distancias_camion

def matriz_distancias_dron(lista_nodos):
    cant_nodos = len(lista_nodos)
    matriz_distancias_dron = [[0 for _ in range(cant_nodos)] for _ in range(cant_nodos)]
    for i in range(cant_nodos):
        for j in range(i + 1, cant_nodos):
            distancia = distancia_dron(lista_nodos[i], lista_nodos[j])
            matriz_distancias_dron[i][j] = matriz_distancias_dron[j][i] = distancia
    return matriz_distancias_dron

def calcular_matrices_costos(matriz_dist_camion, costo_camion, matriz_dist_dron, costo_dron):
    cant_nodos = len(matriz_dist_camion)  # Asumiendo que ambas matrices tienen el mismo tamaño
    matriz_costos_camion = [[0 for _ in range(cant_nodos)] for _ in range(cant_nodos)]
    matriz_costos_dron = [[0 for _ in range(cant_nodos)] for _ in range(cant_nodos)]

    for i in range(cant_nodos):
        for j in range(cant_nodos):
            matriz_costos_camion[i][j] = matriz_dist_camion[i][j] * costo_camion
            matriz_costos_dron[i][j] = matriz_dist_dron[i][j] * costo_dron

    return matriz_costos_camion, matriz_costos_dron




def calcular_matrices_tiempos(matriz_dist_camion, velocidad_camion, matriz_dist_dron, velocidad_dron):
    cant_nodos = len(matriz_dist_camion)
    matriz_tiempos_camion = [[0 for _ in range(cant_nodos)] for _ in range(cant_nodos)]
    matriz_tiempos_dron = [[0 for _ in range(cant_nodos)] for _ in range(cant_nodos)]

    for i in range(cant_nodos):
        for j in range(cant_nodos):
            if matriz_dist_camion[i][j] != 0:
                matriz_tiempos_camion[i][j] = (matriz_dist_camion[i][j] / velocidad_camion) * 3600
            if matriz_dist_dron[i][j] != 0:
                matriz_tiempos_dron[i][j] = (matriz_dist_dron[i][j] / velocidad_dron) * 3600

    return matriz_tiempos_camion, matriz_tiempos_dron

def matriz_feromonas(lista_nodos,f0):
    cant_nodos = len(lista_nodos)
    matriz_feromonas = []
    for i in range(cant_nodos):
        matriz_feromonas.append([0 if i == j else f0 for j in range(cant_nodos)])

    return matriz_feromonas


In [5]:
costo_camion = 1.0  # Ejemplo de costo por km para el camión
costo_dron = 0.5    # Ejemplo de costo por km para el dron
velocidad_camion = 50  # km/h
velocidad_dron = 40   # km/h
f0 = 1 # valor inicial feromonas, un valor random que sea con distribucion uniforme
matriz_dist_camion = matriz_distancias_camion(nodos, G)
matriz_dist_dron = matriz_distancias_dron(nodos)

# 
matriz_costos_camion, matriz_costos_dron = calcular_matrices_costos(
    matriz_dist_camion, costo_camion, matriz_dist_dron, costo_dron
)

matriz_tiempos_camion, matriz_tiempos_dron = calcular_matrices_tiempos(
    matriz_dist_camion, velocidad_camion, matriz_dist_dron, velocidad_dron
)
#libreria 
matriz_feromonas_camion = matriz_feromonas(nodos,f0)#camion  rutas mas cortas, probar que hace la ruta completa, le saco una cordenada y la calculo de nuevo, la complemento con el dron y le sumo el costo de mandar el dron ahi,
#despues le saco otra cordenada, que me devuelva cual es la cantidad en viajes de dron, que en base a eso me arme los csotos, la combinatoria en dron, combinatoria entera, cuantos viajes y cuales, me voy quedando con el menor cosd, un doble for con muchas funciones
matriz_fermonas_dron = matriz_feromonas(nodos,f0)# esta mas abierto no hay calles definidas


### Matriz de Distancias para el camión

In [15]:
mejor_ruta_camion, mejor_ruta_dron, mejor_costo, mejor_tiempo, tiempo = ejecutar(nodos, hormigas, iteraciones, matriz_costos_camion, matriz_costos_dron,  matriz_tiempos_camion, matriz_tiempos_dron, matriz_feromonas_camion, matriz_fermonas_dron,
         alfa, beta, const, max_vuelo, max_capacidad, limite_tiempo, evaporacion)


0


TypeError: float() argument must be a string or a real number, not 'complex'

In [ ]:
mejor_ruta_camion

In [ ]:
mejor_ruta_dron

In [ ]:
import osmnx as ox
import folium
import folium.plugins as plugins

# Configuración inicial
central_lat, central_lon = -33.456574386393314, -70.65970917320084
# Descarga del grafo de OSMnx
G = ox.graph_from_bbox(north, south, east, west, network_type='drive')

# Calcula las rutas para el camión
calculated_route_segments = []
for i in range(len(mejor_ruta_camion) - 1):
    origen = ox.distance.nearest_nodes(G, float(nodos[mejor_ruta_camion[i]]['long']), float(nodos[mejor_ruta_camion[i]]['lat']))
    destino = ox.distance.nearest_nodes(G, float(nodos[mejor_ruta_camion[i + 1]]['long']), float(nodos[mejor_ruta_camion[i + 1]]['lat']))
    route = ox.shortest_path(G, origen, destino, weight='length')
    route_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route]
    calculated_route_segments.append(route_coords)

# Crea el mapa base
m = folium.Map(location=[central_lat, central_lon], zoom_start=14)

# Dibuja las rutas del camión con flechas
for segment in calculated_route_segments:
    plugins.AntPath(locations=segment, color="blue", weight=5, opacity=0.7, delay=1000).add_to(m)

# Dibuja la ruta del drón con flechas
for i in range(len(mejor_ruta_dron) - 1):
    start_coords = (float(nodos[mejor_ruta_dron[i]]['lat']), float(nodos[mejor_ruta_dron[i]]['long']))
    end_coords = (float(nodos[mejor_ruta_dron[i + 1]]['lat']), float(nodos[mejor_ruta_dron[i + 1]]['long']))

    # Verifica si el dron está viajando en el camión en esta sección
    if mejor_ruta_dron[i] in mejor_ruta_camion and mejor_ruta_dron[i + 1] in mejor_ruta_camion:
        # Encuentra el segmento correspondiente de la ruta del camión
        index_camion_start = mejor_ruta_camion.index(mejor_ruta_dron[i])
        index_camion_end = mejor_ruta_camion.index(mejor_ruta_dron[i + 1])

        # Verifica si los índices son consecutivos en la ruta del camión
        if index_camion_end == index_camion_start + 1:
            segmento_camion = calculated_route_segments[index_camion_start]
            plugins.AntPath(locations=segmento_camion, color="purple", weight=5, opacity=0.7, delay=1000, dash_array='5,5').add_to(m)
        else:
            # Dron y camión no están viajando juntos en esta sección
            plugins.AntPath(locations=[start_coords, end_coords], color="red", weight=5, opacity=0.7, delay=1000).add_to(m)
    else:
        # Estilo de línea con flechas para el dron
        plugins.AntPath(locations=[start_coords, end_coords], color="red", weight=5, opacity=0.7, delay=1000).add_to(m)

# Función para añadir marcadores
def add_marker(m, location, popup_text, delivery_type):
    icon_color = "blue" if delivery_type == "truck" else "red"
    icon_type = "truck" if delivery_type == "truck" else "plane"
    folium.Marker(
        location=location,
        popup=popup_text,
        icon=folium.Icon(icon=icon_type, color=icon_color, prefix='fa')
    ).add_to(m)

# Agrega marcadores para los nodos con información de entrega
for i, nodo in enumerate(nodos):
    location = (float(nodo['lat']), float(nodo['long']))
    if i in mejor_ruta_camion:
        delivery_type = 'truck'
    elif i in mejor_ruta_dron:
        delivery_type = 'drone'
    else:
        continue  # Si el nodo no está en ninguna ruta, no añadir marcador

    popup_text = f"Entrega {i}: {delivery_type.capitalize()}"
    add_marker(m, location, popup_text, delivery_type)

# Alamcane
almacen = (float(nodos[0]['lat']), float(nodos[0]['long']))

# Encuentra la ruta de regreso al almacén para el camión
ultimo_nodo_camion = (float(nodos[mejor_ruta_camion[-1]]['lat']), float(nodos[mejor_ruta_camion[-1]]['long']))
origen = ox.distance.nearest_nodes(G, ultimo_nodo_camion[1], ultimo_nodo_camion[0])
destino = ox.distance.nearest_nodes(G, almacen[1], almacen[0])
ruta_regreso = ox.shortest_path(G, origen, destino, weight='length')
ruta_regreso_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in ruta_regreso]

# Dibuja la ruta de regreso con flechas
if mejor_ruta_dron[-1] == mejor_ruta_camion[-1]:
    # Dron y camión regresan juntos
    for i in range(len(ruta_regreso_coords) - 1):
        plugins.AntPath(locations=[ruta_regreso_coords[i], ruta_regreso_coords[i + 1]], color="blue", weight=5, opacity=0.7, delay=1000).add_to(m)
        plugins.AntPath(locations=[ruta_regreso_coords[i], ruta_regreso_coords[i + 1]], color="purple", weight=5, opacity=0.7, delay=1000, dash_array='5,5').add_to(m)
else:
    # Solo el camión regresa
    plugins.AntPath(locations=ruta_regreso_coords, color="blue", weight=5, opacity=0.7, delay=1000).add_to(m)

# Agrega un marcador para el almacén
folium.Marker(
    location=almacen,
    icon=folium.Icon(color="green", icon="home")
).add_to(m)

# Mostrar el mapa
m


In [ ]:
nodos

In [ ]:
mejor_ruta_camion

In [ ]:
route_coords

In [ ]:
mejor_costo

In [ ]:
mejor_tiempo

In [ ]:
tiempo

In [4]:
import folium
import json

almacen = {"id": "0", "lat": "-33.456574386393314", "long": "-70.65970917320084", "demand": 0}

# Datos de los clientes
with open("nodos.json", 'r', encoding='utf-8') as archivo:
    nodos = json.load(archivo)


nodos.insert(0, almacen)  # Asegúrate de que el almacén es el primer nodo en la lista

# Configuración inicial del mapa
central_lat, central_lon = -33.456574386393314, -70.65970917320084
m = folium.Map(location=[central_lat, central_lon], zoom_start=14)

# Función para añadir marcadores
def add_marker(m, location, popup_text, icon_color='blue', icon='info-sign'):
    folium.Marker(
        location=location,
        popup=popup_text,
        icon=folium.Icon(color=icon_color, icon=icon)
    ).add_to(m)

# Agrega marcadores para los nodos
for nodo in nodos:
    location = (float(nodo['lat']), float(nodo['long']))
    popup_text = f"Nodo {nodo['id']}"
    add_marker(m, location, popup_text)

# Agrega un marcador para el almacén (primer nodo)
almacen = (float(nodos[0]['lat']), float(nodos[0]['long']))
add_marker(m, almacen, 'Almacén', 'green', 'home')

# Mostrar el mapa
m

In [ ]:
import sys
def ejecutar_algoritmo(alfa, beta, const):
    # Define los valores de los parámetros que necesitas para tu algoritmo
    hormigas = 1
    iteraciones = 100
    max_vuelo = 10 * 60  # 10 minutos convertidos a segundos
    max_capacidad = 1
    limite_tiempo = 1
    evaporacion = 0.02

    # Aquí asumo que 'nodos' y las matrices son definidas globalmente o importadas
    mejor_ruta_camion, mejor_ruta_dron, mejor_costo, mejor_tiempo, tiempo = ejecutar(
        nodos, hormigas, iteraciones, matriz_costos_camion, matriz_costos_dron,
        matriz_tiempos_camion, matriz_tiempos_dron, matriz_feromonas_camion, 
        matriz_feromonas_dron, alfa, beta, const, max_vuelo, max_capacidad, 
        limite_tiempo, evaporacion
    )

    # Asumiendo que quieres devolver el mejor costo como resultado
    return mejor_costo
if __name__ == "__main__":
    # Recibe parámetros desde la línea de comandos
    alfa = float(sys.argv[1])
    beta = int(sys.argv[2])
    const = float(sys.argv[3])

    # Llama a la función de tu algoritmo con los parámetros recibidos
    resultado = ejecutar_algoritmo(alfa, beta, const, ...otros parámetros...)
    
    # Imprime el resultado
    print(resultado)

In [ ]:
import sys

if __name__ == "__main__":
    # Recibe parámetros de iRace
    alfa = float(sys.argv[1])
    beta = int(sys.argv[2])
    const = float(sys.argv[3])

    # Ejecuta el algoritmo con los parámetros recibidos
    resultado = ejecutar_algoritmo(alfa, beta, const)

    # Imprime el resultado para que iRace lo utilice
    print(resultado)

In [ ]:
import subprocess
import os

# Cambiar al directorio donde está iRace y tus archivos
os.chdir("C:\\tesis_manrre\\CODIGO FINAL")

# Comando para ejecutar iRace
comando_irace = "irace --scenario scenario.txt"

# Ejecutar iRace
process = subprocess.Popen(comando_irace, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
output, error = process.communicate()

# Mostrar resultados
print("Salida:", output.decode())
print("Error:", error.decode())


In [ ]:
irace --scenario "C:/tesis_manrre/CODIGO FINAL/scenario.txt"
